In [5]:
import json
from pathlib import Path
import torch
import numpy as np

In [13]:
p_ann_json = Path('/data/gunsbrother/prjs/ltvu/llms/GroundVQA/data/unified/annotations.NLQ_val.json')
anns = json.load(p_ann_json.open())
anns[0]

{'video_id': 'cc2d7790-67f7-4e52-9fa9-33121c9431a2',
 'sample_id': '95441b96-e2ef-4e61-b74f-10ebc2da5b4e_0',
 'question': 'Where did i put the fire gun ?',
 'moment_start_frame': 858.1167,
 'moment_end_frame': 870.3900000000001,
 'clip_start_sec': 28.60389,
 'clip_end_sec': 29.013,
 'clip_duration': 480.0156449908854}

In [17]:
max_ious = []

for ann in anns:
    duration_sec = ann['clip_duration']
    ts = np.arange(0, duration_sec, 10)
    preds = np.stack([ts - 15., ts + 15.], axis=1)
    gt = np.array([ann['clip_start_sec'], ann['clip_end_sec']])

    inters = np.maximum(0, np.minimum(preds[:, 1], gt[1]) - np.maximum(preds[:, 0], gt[0]))
    unions = np.maximum(preds[:, 1], gt[1]) - np.minimum(preds[:, 0], gt[0])
    ious = inters / unions
    max_ious.append(ious.max())
max_ious = np.array(max_ious)
max_ious[:10]

array([0.013637  , 0.05734267, 0.01352567, 0.03523333, 0.02256433,
       0.11907233, 0.02614067, 0.01175667, 0.05799467, 0.02328067])

In [18]:
(max_ious > .3).mean()

0.2592267135325132

In [19]:
(max_ious > .5).mean()

0.14015817223198593